In [9]:
import pandas as pd

from os import path, getcwd

PROCESSED_DATA_PATH = "../data/hydrated_data_ieee/final_data/processed.csv"

df = pd.read_csv(PROCESSED_DATA_PATH)

/Library/Python/3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
import re
import string
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        # for safety
        return wordnet.NOUN
    
sad_emoticons = {":-(", ":(", ":-|", ";-(", ";-<", "|-{"}
happy_emoticons = {":-)", ":)", ":o)", ":-}", ";-}", ":->", ";-)", ":p", ":D", ";p", "XD"}

dct = {
    'gr8': 'great', 
    'awsm': 'awesome',
    'omfg': 'oh my fucking god',
    'omg': 'oh my god',
    'af': 'as fuck',
    'asap': 'as soon as possible',
    'lmao': 'laugh my ass off',
    'approx': 'approximately',
    'appt': 'appointmnt',
    'apt':' apartment',
    'dept': 'department',
    'diy': 'do it yourself',
    'rsvp': 'reserve your spot',
    'plz': 'please',
    'fk': 'fuck',
    'thx': 'thanks',
    'u': 'you',
    'lol': 'laughing out loud'
}


def pre_process_text(text):
    splitted_text = text.split(" ")
    
    result = []

    for i in range(len(splitted_text)):
        if splitted_text[i] in sad_emoticons:
            result.append("sad")
        elif splitted_text[i] in happy_emoticons:
            result.append("happy")
        elif splitted_text[i].startswith("#"):
            capital_seperate_text = splitted_text[i][1: ]
            for word in re.findall('[A-Z][a-z]*', capital_seperate_text):
                result.append(word)
        else:
            result.append(splitted_text[i])
            
    result = " ".join(result)
    # mark lower case
    lowered = result.translate(str.maketrans('', '', string.punctuation)).lower()

    # handle abbreviations from dct above
    lowered = " ".join([dct[i] if i in dct else i for i in lowered.split(" ")])
    # lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(lowered)
    tagged = nltk.pos_tag(tokens)
    
    return " ".join([lemmatizer.lemmatize(_[0], pos=get_wordnet_pos(_[1])) for _ in tagged])

[nltk_data] Downloading package punkt to /Users/tingfengx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tingfengx/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tingfengx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
df.head()

,Unnamed: 0,Unnamed: 0.1,tweet_id,text,retweet_count,favorite_count,possibly_sensitive,withheld_in_countries,user.id,user.name,...,user.listed_count,user.created_at,user.favourites_count,user.statuses_count,user.default_profile,user.default_profile_image,user.withheld_in_countries,date,sentiment_score,expanded_text
0,0,0,1.240728e+18,holy shit I hate every person in this video,70104,0,NaN,NaN,2.022756e+08,Typreme Kicks.👟❤️📈🤙🏽,...,9,Wed Oct 13 17:48:24 +0000 2010,89995,63405,True,False,NaN,Mar 19,-0.5000,holy shit i hate every person in this video
1,1,1,1.240728e+18,There you have it.\n\nProof that this is an in...,1662,0,NaN,NaN,3.524761e+08,Jake,...,3,Wed Aug 10 17:37:59 +0000 2011,9823,21872,False,False,NaN,Mar 19,0.0000,there you have it proof that this be an intent...
2,2,2,1.240728e+18,"Among other things, what this shows is they're...",6624,0,NaN,NaN,9.281008e+17,Fifty Pound Head,...,0,Wed Nov 08 03:24:12 +0000 2017,1651,1617,False,False,NaN,Mar 19,-0.0625,among other thing what this show be they be us...
3,3,3,1.240729e+18,this incompetent racist asshole,14445,0,False,NaN,1.585784e+07,Brad,...,2,Fri Aug 15 00:33:06 +0000 2008,8538,11553,False,False,NaN,Mar 19,-0.3500,this incompetent racist asshole
4,4,4,1.240729e+18,"Among other things, what this shows is they're...",6624,0,NaN,NaN,1.001945e+18,Pamela ( No DMs),...,3,Wed May 30 21:56:16 +0000 2018,190490,99423,True,False,NaN,Mar 19,-0.0625,among other thing what this show be they be us...


In [12]:
df["expanded_text"] = df["expanded_text"].map(pre_process_text)

In [13]:
# save the preprocessed dataframe (for later evaluation)
df.to_csv(PROCESSED_DATA_PATH)